In [ ]:
import numpy as np

In [111]:
import pandas as pd

In [1]:
import os

In [2]:
import re

In [ ]:
pd.Series([True,False,True,True,False,True,True,False,True,True,False,True]).sum()

In [ ]:
data = {'Name':['Tom','James','Ricky','Vin','Steve','Minsu','Jack'],
        'Age':[25,26,25,23,30,29,23],
        'Rating':[4.23,3.24,3.98,2.56,3.20,4.6,3.8]
        }

df = pd.DataFrame(data)

In [ ]:
df

In [ ]:
people = pd.Series([100,100],index=[1,3],name='new_Rating')

In [ ]:
people

In [ ]:
df.Rating * people

In [ ]:
df.info(memory_usage='deep')

In [ ]:
drinks = pd.read_csv('https://bit.ly/drinksbycountry')

In [ ]:
drinks.head()

In [ ]:
drinks.info()

In [ ]:
drinks.info(memory_usage='deep')

In [ ]:
drinks.memory_usage()

In [ ]:
drinks.memory_usage(deep=True)

In [ ]:
drinks.memory_usage(deep=True).sum()

In [ ]:
df = drinks.loc[:,['beer_servings','continent']]

In [ ]:
df.head()

In [ ]:
df.memory_usage(deep=True)

In [ ]:
df['continent'] = df.continent.astype('category')

In [ ]:
df.memory_usage(deep=True)

In [ ]:
df.continent.head()

In [ ]:
df.continent.cat.codes.head()

In [ ]:
df.continent.cat.categories

In [ ]:
import os

#切换工作目录
os.chdir('D:\VDownload\新建文件夹\我的解析任务1804241345\合并')

#迭代文件夹和文件名
for root,dirs,files in os.walk('D:\VDownload\新建文件夹\我的解析任务1804241345\合并'):
    #取文件名，处理
    for i in files:
        l = i.split(' ',1)[1].strip()
        print(l)

        #重命名文件
        os.rename(i,l)

In [5]:
for root,dirs,files in os.walk(r'D:\VDownload\NYC Yellow Taxi data'):
    print(files)

['yellow_tripdata_2009-01.csv', 'yellow_tripdata_2009-02.csv', 'yellow_tripdata_2009-03.csv', 'yellow_tripdata_2009-04.csv', 'yellow_tripdata_2013-08.csv', 'yellow_tripdata_2013-09.csv', 'yellow_tripdata_2013-10.csv', 'yellow_tripdata_2013-11.csv', 'yellow_tripdata_2014-06.csv', 'yellow_tripdata_2014-08.csv', 'yellow_tripdata_2015-02.csv', 'yellow_tripdata_2015-03.csv', 'yellow_tripdata_2015-10.csv', 'yellow_tripdata_2016-05.csv', 'yellow_tripdata_2016-06.csv', 'yellow_tripdata_2016-07.csv', 'yellow_tripdata_2016-08.csv', 'yellow_tripdata_2016-10.csv', 'yellow_tripdata_2016-11.csv', 'yellow_tripdata_2016-12.csv', 'yellow_tripdata_2017-01.csv', 'yellow_tripdata_2017-03.csv', 'yellow_tripdata_2017-04.csv', 'yellow_tripdata_2017-06.csv', 'yellow_tripdata_2017-07.csv', 'yellow_tripdata_2017-08.csv', 'yellow_tripdata_2017-09.csv', 'yellow_tripdata_2017-10.csv', 'yellow_tripdata_2017-11.csv', 'yellow_tripdata_2017-12.csv', 'yellow_tripdata_2018-01.csv', 'yellow_tripdata_2018-02.csv', 'yellow

In [85]:
name = os.listdir(r'D:\VDownload\NYC Yellow Taxi data')

In [9]:
type(files)

list

In [102]:
len(name)

36

In [11]:
'x' in files

False

In [12]:
'yellow_tripdata_2009-01.csv' in files

True

In [57]:
add = open(r'C:\Users\Xiaoji\Desktop\1024.txt')

In [16]:
list(map(lambda x:print(x),add))

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-05.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-06.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-07.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-08.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-09.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-10.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-11.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-12.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-01.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-02.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-03.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-04.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/ye

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [58]:
content = add.readlines()

In [109]:
l = []
for i in content:
    print(i)
    match = re.search(r'(/)([fgy](.*))',i)
    if match:
        if match.group(2) not in name:
            l.append(i)
        print('ok')
        l.append(i)

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-05.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-06.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-07.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-08.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-09.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-10.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-11.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-12.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-01.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-02.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-03.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-04.csv

Yellow	https://s3.amazonaws.com/nyc-tlc/trip+data/ye

In [108]:
l

['Yellow\thttps://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-05.csv\n',
 'Yellow\thttps://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-06.csv\n',
 'Yellow\thttps://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-07.csv\n',
 'Yellow\thttps://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-08.csv\n',
 'Yellow\thttps://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-09.csv\n',
 'Yellow\thttps://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-10.csv\n',
 'Yellow\thttps://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-11.csv\n',
 'Yellow\thttps://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-12.csv\n',
 'Yellow\thttps://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-01.csv\n',
 'Yellow\thttps://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-02.csv\n',
 'Yellow\thttps://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-03.csv\n',
 'Yellow\thttps://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripd

In [101]:
len(l)

64

In [104]:
line = 'Green	https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-01.csv'

In [105]:
match = re.search(r'(/)([fgy](.*)[v$])',line)

In [106]:
match.group(2) not in name

False

In [110]:
add.close()

In [115]:
%timeit df = pd.read_csv(r'C:\Users\Xiaoji\Downloads\green_tripdata_2013-08.csv')

42.8 ms ± 2.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [114]:
%timeit df

30.3 ns ± 1.58 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [117]:
df = pd.read_csv(r'D:\VDownload\NYC Yellow Taxi data\yellow_tripdata_2009-03.csv')

MemoryError: 

In [ ]:
%timeit df = pd.read_csv(r'D:\VDownload\NYC Yellow Taxi data\yellow_tripdata_2009-03.csv')